# Monitoramento de casos de CVID19 no Brasil.

**Este notebook tem como objetivo aplicar técnicas de análise da dados, somente para fins didáticos.**

### INFORMAÇÕES SOBRE AS BASES UTILIZADAS

* Base utilizada para extrair dados do Brasil e Estados [G1  Globo](https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/27/casos-de-coronavirus-no-brasil-em-27-de-marco.ghtml) Fonte: Ministério da Saúde até 15 de março; secretarias estaduais da Saúde a partir de 16 de março.Criado com Datawrapper

* Base utilizada para extrair dados dos Municipios [brasil.io](https://brasil.io/dataset/covid19/caso) Boletins informativos baseado no Ministério da Saúde e secretarias estaduais da Saúde.

* Base utilizada para extrair dados dos Mundial  [BNO News]( https://bnonews.com/index.php/2020/03/the-latest-coronavirus-cases ) Boletins informativos baseado nos dados da OMS.

* Base utilizada para extrair dados dos Mundial [OMS]( https://who.sprinklr.com)OMS.

* Base utilizada para extrair dados dos Estados [Ministério da Saúde]( https://covid.saude.gov.br/ )

*Nenhuma copia deste notebook é permitida sem citar a fonte original.*

In [67]:
import pandas as pd
import numpy as np
import cufflinks as cf
import wget
import plotly
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import matplotlib.pyplot as plt
import folium
import os
import requests
import json
from pandas.io.json import json_normalize
from folium import plugins
from numpy import nan as NA
from plotly.offline import iplot
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
cf.go_offline()
plotly.offline.init_notebook_mode(connected=True)

## Carregamento e Tratamento dos Dados

In [68]:
acumulado_dia_brasil = pd.read_csv('DB/data-43mJk.csv', sep=',' ,index_col=None)
acumulado_estados = pd.read_csv('DB/data-eXkcW.csv', sep=',' ,index_col=None)
estados = pd.read_csv('DB/estados.csv', encoding='utf-8')
municipios = pd.read_csv('DB/municipios.csv', encoding='utf-8')
api_br_io = pd.read_csv('DB/api_br_io.csv')
api_br_io = api_br_io.fillna(0)
api_br_io.city_ibge_code = api_br_io.city_ibge_code.astype('int64')

url = 'https://brasil.io/dataset/covid19/caso?format=csv'
if os.path.exists('DB/api_br_io.csv'):
    os.remove('DB/api_br_io.csv')
    wget.download(url, 'DB/api_br_io.csv')
else:
    wget.download(url, 'DB/api_br_io.csv')
    
url = 'DB/estados.json'
cidades = pd.read_json(url)
municipios_map = json_normalize(cidades.features)  

########################################################################
#renomeando colunas
acumulado_dia_brasil.columns = ['data', 'casos', 'mortes','coluna1']
municipios.columns = ['city_ibge_code','city','latitude','longitude','capital','uf_code']

#tratando nomes para fazer o merge
estados = estados[['uf', 'nome']]
estados.columns = ['state','state_name']
cidades = api_br_io.merge(municipios.drop('uf_code', axis=1), on=['city','city_ibge_code'], how='left')
cidades = cidades.merge(estados, on='state', how='left')

cidades = cidades[(cidades.is_last==True) & (cidades.city !='Importados/Indefinidos')]
dados = cidades[ cidades.latitude.notnull() ]
cidades = dados[dados.place_type == 'city']

-1 / unknown

C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



### Criando as Regiões do Brasil

In [69]:
#Região Norte
norte = acumulado_estados.groupby(['Estado'])['AC','AP','AM','PA','RO','RR','TO'].agg('sum')
norte['Norte'] = norte.sum(axis=1)
#Região Nordeste
nordeste = acumulado_estados.groupby(['Estado'])['AL','BA','CE','MA','PB','PE','PI','RN','SE'].agg('sum')
nordeste['Nordeste'] = nordeste.sum(axis=1)
#Região Centro-Oeste
centroOeste = acumulado_estados.groupby(['Estado'])['DF','GO','MT','MS'].agg('sum')
centroOeste['CentroOeste'] = centroOeste.sum(axis=1)
#Região Sudeste
sudeste = acumulado_estados.groupby(['Estado'])['ES','MG','RJ','SP'].agg('sum')
sudeste['Sudeste'] = sudeste.sum(axis=1)
#região Sul
sul = acumulado_estados.groupby(['Estado'])['PR','SC','RS'].agg('sum')
sul['Sul'] = sul.sum(axis=1)


C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\joner\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



# Gráficos Brasil

In [70]:
# Plot confirmed cases

fig = go.Figure(
    [
        go.Scatter(x = acumulado_dia_brasil.data, y = acumulado_dia_brasil.casos, mode="lines+text",
    name="Confirmados",
    text=acumulado_dia_brasil.casos,
    textposition="top center"),
        go.Scatter(x = acumulado_dia_brasil.data, y = acumulado_dia_brasil.mortes, mode="lines+text",
    name="Mortes",
    text=acumulado_dia_brasil.mortes,
    textposition="top center"), 
      #  go.Scatter(x = dadosdia.data, y = curados, mode="lines+text",
   # name="Curados",
   # text=curados,
   # textposition="top center"),  
    ]
)

fig.update_layout(title='Casos acumulados por dia',legend_title='<b> </b>',
                   
                   showlegend=True, template="plotly_white",direction='clockwise',)

fig.show()

![Figura1 Acumulado dos casos no Brasil por dia](IMG/AcumuladoBrasilDia.png)

In [71]:
# Plot confirmed cases
fig = go.Figure(
    [
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.SP, mode = 'markers+lines', name="São Paulo", line_shape='spline'),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.RJ, mode = 'markers+lines', name="Rio de Janeiro", line_shape='spline'),
         go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.RS, mode = 'markers+lines', name="Rio Grande do Sul", line_shape='spline'),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.DF, mode = 'markers+lines', name="Distrito Federal", line_shape='spline'),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.CE, mode = 'markers+lines', name="Ceará"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.GO, mode = 'markers+lines', name="Goiás"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.AC, mode = 'markers+lines', name="Acre"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.AL, mode = 'markers+lines', name="Alagoas"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.AM, mode = 'markers+lines', name="Amazonas"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.AP, mode = 'markers+lines', name="Amapá"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.BA, mode = 'markers+lines', name="Bahia"),
        
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.ES, mode = 'markers+lines', name="Espírito Santo"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.MA, mode = 'markers+lines', name="Maranhão"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.MT, mode = 'markers+lines', name="Mato Grosso"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.MS, mode = 'markers+lines', name="Mato Grosso do Sul"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.MG, mode = 'markers+lines', name="Minas Gerais"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.PA, mode = 'markers+lines', name="Pará"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.PB, mode = 'markers+lines', name="Paraíba"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.PR, mode = 'markers+lines', name="Paraná"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.PE, mode = 'markers+lines', name="Pernambuco"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.PI, mode = 'markers+lines', name="Piauí"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.RN, mode = 'markers+lines', name="Rio Grande do Norte"),
        
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.RO, mode = 'markers+lines', name="Rondônia"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.RR, mode = 'markers+lines', name="Roraima"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.SC, mode = 'markers+lines', name="Santa Catarina"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.SE, mode = 'markers+lines', name="Sergipe"),
        go.Scatter(x = acumulado_estados.Estado, y = acumulado_estados.TO, mode = 'markers+lines', name="Tocantins"),
    ]
)

fig.update_layout(title='Casos acumulados por Estado',legend_title='<b> Estados/DF </b>',
                   
                   showlegend=True, template="plotly_white",direction='clockwise')

fig.show()

![Figura2 Acumulado dos casos no Brasil por dia](IMG/AcumuladoEstadosDia.png)

In [72]:
labels = ['Norte','Sul','Nordeste','Centro-Oeste','Sudeste']
values = [norte.Norte.max(),sul.Sul.max(), nordeste.Nordeste.max(),centroOeste.CentroOeste.max(),sudeste.Sudeste.max()]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent')])
fig.show()

![Figura3 Percentual dos casos confirmados por região](IMG/regioes.png)

In [73]:
#api_br_io[ (api_br_io.state == 'RS')]

In [74]:
dataRS =api_br_io[ api_br_io.city.notnull() ]
rs = dataRS [ (dataRS.state == 'RS')&(dataRS.date == '2020-04-08') & (dataRS.place_type =='city')].sort_values(by='confirmed',ascending=False)
rs.groupby(['city'] )
rs.fillna(0, inplace = True)
fig = go.Figure(data=[go.Table(
    header=dict(values=['Cidades  % '+ str(round(rs.city.size*100/497,2)) +'<br><b>Total--</b>'+str(rs.city.size) +'</br>' ,
                        '<br>Confirmados</br> <b>Total-</b>'+ str(rs.confirmed.sum()),
                        '<br>Mortes</br>  <b>Total-</b>'+ str(rs.deaths.sum().astype(int))],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[rs.city, rs.confirmed,rs.deaths],
               fill_color='lavender',
               align='left'))
])

fig.show()

![Tabela1 casos confirmados por cidades do RS](IMG/Alegrete.png)

In [75]:
import plotly.express as px


fig = px.bar(rs.head(10), y='city', x='confirmed',color='deaths', text='confirmed',orientation='h')

fig.update_traces( textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

![Figura4 Acumulado dos casos no RS e óbitos](IMG/RS.png)

In [76]:
br_estados = 'DB/br_states.json'
geo_json_data = json.load(open(br_estados))


In [77]:
dados= cidades.copy()
df_bsb =dados[( dados.city.notnull()) & (dados.place_type =='city') ]

brasil = folium.Map(
    location=[-15.77972, -47.92972],    # Coordenadas retiradas do Google Maps / Longitude ajustada
    zoom_start=4
)



In [78]:

for _, dados in df_bsb.iterrows():
    folium.Marker(
        location=[dados['latitude'], dados['longitude']],clustered_marker = True,
        popup= dados.city +'<br><b>Casos--</b>'+str(dados.confirmed) +'</br>'+'<br><b>Óbitos--</b>'+str(dados.deaths) +'</br>'+'<br><b>Taxa de Mortalidade--</b>'+str(dados.death_rate) +'</br>',
        icon=folium.Icon(color='red',
                    icon_color='black',
                    icon='exclamation-triangle',
                    prefix='fa')
    ).add_to(brasil)
    
brasil

![Figura5 casos no Brasil por dia](IMG/BrasilCasosDITRI.jpg)

In [79]:
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
         
        'fillColor': 'green',
        'color': 'darkred',
        'weight': 0.5,
    }
).add_to(brasil)
brasil

![Figura6 casos no RS ](IMG/RScasosDISTRI.jpg)